# Notes 6D: Regular Expressions

Regular Expressions (sometimes called regex for short) allows a user to search for a particular pattern within a strings using almost any sort of rule they can come up. For example, finding all capital letters in a string, or finding a phone number in a document. 

Regular expressions are notorious for their seemingly strange syntax. This strange syntax is a byproduct of their flexibility. Regular expressions have to be able to filter out any string pattern you can imagine, which is why they have a complex string pattern format.

Let's begin by explaining how to search for basic patterns in a string!

---
## Searching for Basic Patterns

Consider the following string:

In [ ]:
text = "The person's phone number is 408-555-1234. Call soon!"

We'll start off by trying to find out if the string `"phone"` is inside the text string. Now we could quickly do this with:

In [ ]:
"phone" in text

But let's show the format for regular expressions, because later on we will be searching for patterns that won't have such a simple solution.

In [ ]:
import re

In [ ]:
pattern = "phone"
match = re.search(pattern, text)
match

NOTE: `re.search()` takes a pattern, scans the text, and then returns a `Match` object. If no pattern is found, a `None` is returned (in Jupyter Notebook this just means that nothing is output below the cell):

In [ ]:
pattern = "NOT IN TEXT"
re.search(pattern, text)

Let's take a close look at the `Match` object. Notice the `span`, there is also a `start` and `end` index information.

In [ ]:
match.span()

In [ ]:
match.start(), match.end()

In [ ]:
text[match.start():match.end()]

But what if the pattern occurs more than once?

In [ ]:
text = "my phone is a new phone"
match = re.search("phone", text)
match.span()

Notice it only matches the first instance. If we wanted a list of all matches, we can use `re.findall` method:

In [ ]:
matches = re.findall("phone", text)
matches

To get actual match objects, use the iterator:

In [ ]:
[match for match in re.finditer("phone", text)]

If you wanted the actual text that matched, you can use the `group` method.

In [ ]:
match.group()

---
## Patterns

So far we've learned how to search for a basic string. What about more complex examples? Such as trying to find a telephone number in a large string of text? Or an email address?

We could just use search method if we know the specific phone or email, but what if we don't know it? We can look for a general format and we can use that along with regular expressions to search the document for strings that match a particular pattern.

This is where the syntax may appear strange at first, but take your time with this, often its just a matter of looking up the pattern code.

Let' begin!

### Identifiers for Characters in Patterns

Characters such as a digit or a whitespace have different codes that represent them. You can use these to build up a pattern string. Notice how these make heavy use of the backwards slash `\` . Because of this when defining a pattern string for regular expression we use the format:

```python
r"mypattern"
```
    
Placing the `r` in front of the string allows python to understand that the `\` in the pattern string are not meant to be escape slashes.

Below you can find a table of all the possible identifiers:

| Character | Description      | Example Pattern Code | Example Match |
| --------- | ---------------- | -------------------- | ------------- |
| `\d`      | Digit            | `file_\d\d`          | `file_25`     |
| `\w`      | Alphanumeric     | `\w-\w\w\w`          | `A-b_1`       |
| `\s`      | Whitepace        | `a\sb\s\c`           |  `a b c`      |
| `\D`      | Non-digit        | `\D\D\D`             | `ABC`         |
| `\W`      | Non-alphanumeric | `\W\W\W\W\W`         | `*-_=)`       |
| `\S`      | Non-whitespace   | `\S\S\S\S`           | `Yoyo`        |

For example:

In [ ]:
text = "My telephone number is 408-555-1234"
phone = re.search(r"\d\d\d-\d\d\d-\d\d\d\d", text)
phone.group()

Notice the repetition of `\d`. That is a bit of an annoyance, especially if we are looking for very long strings of numbers. Let's explore the possible quantifiers.

### Quantifiers

Now that we know the special character designations, we can use them along with quantifiers to define how many we expect.

| Character | Description                 | Example Pattern Code | Example Match   |
| --------- | --------------------------- | -------------------- | --------------- |
| `+`       | Occurs one or more times    | `Version \w-\w+`     | `Version-Ab1_1` |
| `{n}`     | Occurs exactly `n` times    | `\D{3}`              | `abc`           |
| `{n,m}`   | Occurs `n` to `m` times     | `\d{2,4}`            | `123`           |
| `{n,}`    | Occurs `n` or more times    | `\w{3,}`             | `anycharacters` |
| `*`       | Occurs zero or more times   | `A*B*C*`             | `AAACC`         |
| `?`       | Occurs zero or one times    | `plurals?`           | `plural`        |

Let's rewrite our,previous pattern using these quantifiers:

In [ ]:
re.search(r"\d{3}-\d{3}-\d{4}", text)

### Groups

What if we wanted to do two tasks, find phone numbers, but also be able to quickly extract their area code (the first three digits). We can use groups for any general task that involves grouping together regular expressions (so that we can later break them down). 

Using the phone number example, we can separate groups of regular expressions using parenthesis:

In [ ]:
phone_pattern = re.compile(r"(\d{3})-(\d{3})-(\d{4})")
results = re.search(phone_pattern,text)

In [ ]:
# The entire result
results.group()

In [ ]:
# Can then also call by group position.
# remember groups were separated by parenthesis ()
# Something to note is that group ordering starts at 1. Passing in 0 returns everything
results.group(1)

In [ ]:
results.group(2)

In [ ]:
results.group(3)

In [ ]:
# We only had three groups of parenthesis
results.group(4)

### Additional Regex Syntax

#### `|`: or operator

Use the pipe operator `|` to have an **or** statment. For example:

In [ ]:
re.search(r"man|woman", "This man was here.")

In [ ]:
re.search(r"man|woman", "This woman was here.")

#### `.`: wildcard

Use a "wildcard" as a placement that will match any character placed there. You can use a simple period `.` for this. For example:

In [ ]:
re.findall(r".at", "The cat in the hat sat here.")

In [ ]:
re.findall(r".at", "The bat went splat")

Notice how we only matched the first 3 letters, that is because we need a `.` for each wildcard letter. Or use the quantifiers described above to set its own rules.

In [ ]:
re.findall(r"...at", "The bat went splat")

However this still leads the problem to grabbing more beforehand. Really we only want words that end with `"at"`.

In [ ]:
# One or more non-whitespace that ends with "at"
re.findall(r"\S+at", "The bat went splat")

#### Starts with and Ends With

We can use `^` to signal starts with, and `$` to signal ends with:

In [ ]:
# Ends with a number
re.findall(r"\d$", "This ends with a number 2")

In [ ]:
# Starts with a number
re.findall(r"^\d", "1 is the loneliest number.")

Note that this is for the entire string, not individual words!

#### Exclusion

To exclude characters, we can use the `^` symbol **in conjunction with** a set of square brackets `[]`. Anything inside the brackets is excluded. For example:

In [ ]:
phrase = "there are 3 numbers 34 inside 5 this sentence."

In [ ]:
re.findall(r"[^\d]", phrase)

To get the words back together, use a + sign 

In [ ]:
re.findall(r"[^\d]+", phrase)

We can use this to remove punctuation from a sentence.

In [ ]:
test_phrase = "This is a string! But it has punctuation. How can we remove it?"
re.findall("[^!.? ]+", test_phrase)

In [ ]:
" ".join(re.findall("[^!.? ]+", test_phrase))

### Brackets for Grouping

As we showed above we can use brackets to group together options, for example if we wanted to find hyphenated words:

In [ ]:
text = "Only find the hypen-words in this sentence. But you do not know how long-ish they are"
re.findall(r"[\w]+-[\w]+",text)

### Parenthesis for Multiple Options

If we have multiple options for matching, we can use parenthesis to list out these options. For Example:

In [ ]:
# Find words that start with cat and end with one of these options: "fish", "nap", or "claw"
pattern = "cat(fish|nap|claw)"

In [ ]:
re.search(pattern, "Hello, would you like some catfish?")

In [ ]:
re.search(pattern, "Hello, would you like to take a catnap?")

In [ ]:
# None returned
re.search(pattern, "Hello, have you seen this caterpillar?")

---
## Appendix

For full information on all possible patterns, check out: https://docs.python.org/3/howto/regex.html